<img src="images/cads-logo.png" style="height: 100px;" align=left> <img src="images/sklearn-logo.png" style="height: 100px;" align=right>
# Machine Learning 

# Day 3: k-Nearest Neighbours Classification

Welcome to KNN!

``k-Nearest Neighbours (KNN)`` falls in the **supervised learning** family of algorithms. Informally, this means that we are given a labelled dataset consiting of training observations $(x,y)$ and would like to capture the relationship between $x$ and $y$. More formally, our goal is to learn a function $h: X \rightarrow Y$ so that given an unseen observation $x$, $h(x)$ can confidently predict the corresponding output $y$.

KNN will compare the distance of a new test datapoint to the datapoints in the train set and then decide among the K nearest neighbors which class the new data point will be!

## Table of Contents

- [The KNN Algorithm](#The-KNN-Algorithm)
- [Distances](#Distances)
    - [What is a distance](#What-is-a-distance)
    - [What distances do you know](#What-distances-do-you-know)
    - [Minkowski distance](#Minkowski-distance)
- [Normalization](#Normalization)
- [KNN Applied](#KNN-Applied)
    - [Load the fruits data file](#Load-the-fruits-data-file)
    - [Examining the data](#Examining-the-data)
    - [Create train-test split](#Create-train-test-split)
    - [Create classifier object](#Create-classifier-object)
    - [Train the classifier (fit the estimator) using the training data](#Train-the-classifier-(fit-the-estimator)-using-the-training-data)
    - [Estimate the accuracy of the classifier on future data, using the test data](#Estimate-the-accuracy-of-the-classifier-on-future-data,-using-the-test-data)
    - [Use the trained k-NN classifier model to classify new and previously unseen objects](#Use-the-trained-k-NN-classifier-model-to-classify-new-and-previously-unseen-objects)
    - [How sensitive is k-NN classification accuracy to the choice of the 'k' parameter](#How-sensitive-is-k-NN-classification-accuracy-to-the-choice-of-the-'k'-parameter)
    - [How sensitive is k-NN classification accuracy to the train/test split proportion?](#How-sensitive-is-k-NN-classification-accuracy-to-the-train/test-split-proportion?)
    - [Pros and Cons of KNN](#Pros-and-Cons-of-KNN)
    - [Exercise - KNN on the Iris dataset](#Exercise---KNN-on-the-Iris-dataset)
    - [Exercise - KNN on Wine Quality](#Exercise---KNN-on-Wine-Quality)


## The KNN Algorithm

NN will compare the distance of a new test datapoint to the datapoints in the train set and then decide among the K nearest neighbors which class the new data point will be!

 <img src='images/knn2.jpg'/>
 
Each test sample is classified using majority vote of its neighbors by the following steps:
   - 1) Distances from the test sample to all stored training sample are calculated using a specific distance function or similarity measure.
   - 2) The K nearest neighbors of the test sample are selected, where K is a pre-defined small integer.
   - 3) The most repeated class of these K neighbors is assigned to the test sample. In other words, a test sample is assigned to the class c if it is the most frequent class label among the K nearest training samples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

sns.set()
%matplotlib inline

## Distances

Since KNN determine the distance between datapoints it is a good start to determin what exactly is a distance and how can we calculate the distance between two points.

### What is a distance

**Question:** What are the 5 properties of a distance?
<br><br><br><br><br><br><br><br><br><br><br><br>
**Answer:**

### What distances do you know

**Question:** What distance metrics do you know?
<br><br><br><br><br><br>
**Answer:**

### Minkowski distance

The Minkowski distance is a metric in a normed vector space which can be considered as a generalization of both the Euclidean distance and the Manhattan distance.

#### Manhattan distance (p=1): 
$$ \sum_{i=1}^n |x_i-y_i|$$

#### Euclidean distance (p=2): 
$$ \sqrt{\sum_{i=1}^n (x_i-y_i)^2} $$

#### Minkowski distance: 
$$ \left(\sum_{i=1}^n |x_i-y_i|^p\right)^{1/p} $$

<br>

The euclidean distance is a straight line between two points in a 2 dimensional space.
The manhattan distance is a absolute value of the two differences. 

## Normalization

Below and example of how different scales can make one feature more dominant than the other. To give the features equal importance we can apply normalization.

In [ ]:
# woman 1.50m and 65kg
woman = (1.50, 65)

# man 1.90m and 80kg
man = (1.90, 80)

# unknown gender 1.80m and 70kg
x = (1.80, 70)

In [ ]:
def euclidian(pers1, pers2):
    h1, w1 = pers1
    h2, w2 = pers2
    return ((h2-h1)**2 + (w2-w1)**2)**(0.5)


print("distance to woman:", euclidian(x, woman))
print("distance to man:", euclidian(x, man))

Now we normalize using MinMax scaling:

$$ X'=\frac{X - X_{min}}{X_{max}-X_{min}}$$

In [ ]:
# lets say these are the smallest and largests value found in the data
max_h = 2
min_h = 1.4
max_w = 100
min_w = 50

def normalize(pers):
    h, w = pers
    h_norm = (h-min_h)/(max_h-min_h)
    w_norm = (w-min_w)/(max_w-min_w)
    return h_norm, w_norm

print("")
print("normalized distance to woman:", euclidian(normalize(x), normalize(woman)))
print("normalized distance to man:", euclidian(normalize(x), normalize(man)))

In [ ]:
from sklearn.preprocessing import MinMaxScaler , StandardScaler

x = [[1.50, 65], [1.90, 80], [1.80, 70], [1.4, 50], [2, 100]] 
# x[0]=woman,  x[1]= man,  x[2]= unknown gender,  x[3]= min,  x[4] = max    

scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)          # Fit to data, then transform it

print("")
print("normalized distance to woman:", euclidian(x_scaled[2], x_scaled[0] ))
print("normalized distance to man:", euclidian(x_scaled[2], x_scaled[1]))

## KNN Applied 

### Load the fruits data file

In [ ]:
fruits = pd.read_table('fruit_data_with_colors.txt')
fruits.head()

The file contains the mass, height, and width of a selection of oranges, lemons and apples. The heights were measured along the core of the fruit. The widths were the widest width perpendicular to the height.

### Examining the data

In [ ]:
fruits.describe()

In [ ]:
fruits.info()

In [ ]:
fruits.columns

In [ ]:
fruits.dtypes

In [ ]:
# class distribution
print(fruits.groupby('fruit_label').size())     

In [ ]:
# box and whisker plots
fruits.iloc[:, 3:7].plot(kind='box', subplots=True, layout=(2,2))
plt.show()

In [ ]:
# histograms
fruits.iloc[:, 3:7].hist()
plt.show()

In [ ]:
sns.pairplot(fruits, kind="scatter", hue="fruit_name",vars=["mass", "width", "height", "color_score"], markers=["o", "s", "D", '+'], palette="Set2")
plt.show() 
# palette: Set of colors for mapping the hue variable


### Create train-test split

In [ ]:
# For this example, we use the mass, width, and height features of each fruit instance
X = fruits[['mass', 'width', 'height']]
y = fruits['fruit_label']  # 1: 'apple', 2: 'mandarin', 3: 'orange', 4: 'lemon'

print(X.shape)
print(y.shape)

In [ ]:
# default is 75% / 25% train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
print(X_train[:5])

In [ ]:
print(y_train[:5])

### Create classifier object

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 5)

### Train the classifier (fit the estimator) using the training data

In [ ]:
knn.fit(X_train, y_train)

### Estimate the accuracy of the classifier on future data, using the test data

In [ ]:
knn.score(X_train, y_train)

In [ ]:
knn.score(X_test, y_test)

### Use the trained k-NN classifier model to classify new and previously unseen objects

In [ ]:
# first example: a small fruit with mass 20g, width 4.3 cm, height 5.5 cm

fruit_prediction = knn.predict([[20, 4.3, 5.5]])
fruit_prediction[0]     # 1: 'apple', 2: 'mandarin', 3: 'orange', 4: 'lemon'

In [ ]:
# second example: a larger, elongated fruit with mass 100g, width 6.3 cm, height 8.5 cm

fruit_prediction = knn.predict([[100, 6.3, 8.5]])
fruit_prediction[0]

### How sensitive is k-NN classification accuracy to the choice of the 'k' parameter

In [ ]:
X_train.shape

In [ ]:
k_range = range(1,20)
scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors = k)
    knn.fit(X_train, y_train)
    scores.append(knn.score(X_test, y_test))

plt.figure()
plt.xlabel('k')
plt.ylabel('accuracy')
plt.scatter(k_range, scores)
plt.xticks([0,5,10,15,20]);

### How sensitive is k-NN classification accuracy to the train/test split proportion?

In [ ]:
t = [0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2]

knn = KNeighborsClassifier(n_neighbors = 5)

plt.figure()

for s in t:

    scores = []
    for i in range(1,1000):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 1-s)
        knn.fit(X_train, y_train)
        scores.append(knn.score(X_test, y_test))
    plt.plot(s, np.mean(scores), 'bo')

plt.xlabel('Training set proportion (%)')
plt.ylabel('accuracy');

### Pros and Cons of KNN

**Pros**

As you can already tell from the previous section, one of the most attractive features of the K-nearest neighbor algorithm is that is simple to understand and easy to implement. With zero to little training time, it can be a useful tool for off-the-bat analysis of some data set you are planning to run more complex algorithms on. Furthermore, KNN works just as easily with multiclass data sets whereas other algorithms are hardcoded for the binary setting. Finally, as we mentioned earlier, the non-parametric nature of KNN gives it an edge in certain settings where the data may be highly “unusual”.

**Cons**

One of the obvious drawbacks of the KNN algorithm is the computationally expensive testing phase which is impractical in industry settings. Note the rigid dichotomy between KNN and the more sophisticated Neural Network which has a lengthy training phase albeit a very fast testing phase. Furthermore, KNN can suffer from skewed class distributions. For example, if a certain class is very frequent in the training set, it will tend to dominate the majority voting of the new example (large number = more common). Finally, the accuracy of KNN can be severely degraded with high-dimension data because there is little difference between the nearest and farthest neighbor.

**Improvements**

With that being said, there are many ways in which the KNN algorithm can be improved.

- A simple and effective way to remedy skewed class distributions is by implementing **weighted voting**. The class of each of the K neighbors is multiplied by a weight proportional to the inverse of the distance from that point to the given test point. This ensures that nearer neighbors contribute more to the final vote than the more distant ones.
- **Changing the distance metric** for different applications may help improve the accuracy of the algorithm. (i.e. Hamming distance for text classification)
- **Rescaling your data** makes the distance metric more meaningful. For instance, given 2 features $height$ and $weight$, an observation such as $x=[180,70]$ will clearly skew the distance metric in favor of height. One way of fixing this is by column-wise subtracting the mean and dividing by the standard deviation. Scikit-learn’s normalize() method can come in handy.
- **Dimensionality reduction** techniques like PCA should be executed prior to appplying KNN and help make the distance metric more meaningful.
- **Approximate Nearest Neighbor** techniques such as using k-d trees to store the training observations can be leveraged to decrease testing time. Note however that these methods tend to perform poorly in high dimensions (20+).

### Exercise - KNN on the Iris dataset

Load the ``iris dataset`` 

Perform the following tasks:
- do some exploratory data analysis on iris dataset
- divide data into 80% train and 20% test
- train and test a KNN model with 3 neighbors
- show accuracy scores for train and test

In [ ]:
import seaborn as sns
iris = sns.load_dataset('iris')
iris.head()

In [ ]:
# Your code here

### Exercise - KNN on Wine Quality

Read the Wine Quality White dataset (``winequality-white.csv``) from your data folder.

Perform the following tasks:
- Select features to be the first 11 columns while the last column to be the target
- Plot the pair plot and boxplot for all features
- Separate the data to 80:20 train and test datasets
- Create an instance of Neighbours Classifier and fit the data (n_neightbors = 5) on the train data
- Measure the accurary for the fitted model on the train data and test data with scaling and without scaling
- Show the accuracy of the KNN models with different number of neighbours on train and test data. ``n_neighbors: [3, 5, 10, 20, 40, 60, 80, 100, 120, 140]`` 


In [ ]:
# Your code here